In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.applications import NASNetMobile


In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
TEST_DIR = "C:\\Users\\sreel\\Downloads\\vs code\\sreelal\\veg-fru\\archive\\test"
TRAIN_DIR ="C:\\Users\\sreel\\Downloads\\vs code\\sreelal\\veg-fru\\archive\\train"
VALID_DIR ="C:\\Users\\sreel\\Downloads\\vs code\\sreelal\\veg-fru\\archive\\validation"

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(TRAIN_DIR,
                                                               seed=2509,
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(VALID_DIR,
                                                               seed=2509,
                                                               image_size=(img_height, img_width),
                                                               shuffle=False,
                                                               batch_size=batch_size)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(TEST_DIR,
                                                              seed=2509,
                                                              image_size=(img_height, img_width),
                                                              shuffle=False,
                                                              batch_size=batch_size)
     

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
base_model=tf.keras.applications.NASNetMobile(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax")

In [ ]:
base_model.trainable = False

In [ ]:
inputs = Input(shape=(224, 224, 3))
x = Rescaling(1./255)(inputs)
x = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(x)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)(x)
x = tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)(x)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(len(class_names), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


In [ ]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(lr=0.001),
    metrics = ["accuracy"])


In [ ]:
model.summary()

In [ ]:
history = model.fit(x=train_ds,
                    epochs= 10,
                    validation_data=valid_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
base_model.trainable = True

In [ ]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(1e-5),
    metrics = ["accuracy"])

In [ ]:

history_fine = model.fit(train_ds,
                         epochs=20,
                         initial_epoch=history.epoch[-1],
                         validation_data=valid_ds)

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.0, 1])
plt.plot([9.9],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
predictions = model.predict(valid_ds, verbose=1)

In [ ]:
predictions.shape

In [ ]:
np.sum(predictions[0])

In [ ]:
predictions[0]

In [ ]:
class_names[np.argmax(predictions[123])]

In [ ]:
score = tf.nn.softmax(predictions[0])
score

In [ ]:
from tensorflow.keras.preprocessing import image
img_path = 'C:\\Users\\sreel\\Downloads\\vs code\\sreelal\\veg-fru\\archive\\validation\\tomato\\Image_1.jpg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(img_height, img_width))

# Convert the image to a NumPy array
img_array = image.img_to_array(img)

# Expand the dimensions to match the input shape expected by the model
img_array = np.expand_dims(img_array, axis=0)
img_array = keras.applications.resnet50.preprocess_input(img_array)


In [ ]:
predictions1 = model.predict(img_array)

In [ ]:
predictions1

In [ ]:
class_names[np.argmax(predictions1)]

In [ ]:
model.save("flower_vegetable_detection_model.h5")